In [1]:
import pandas as pd

In [277]:
df = pd.read_csv('repo/bgg-19m-reviews.csv')

In [278]:
df.columns

Index(['Unnamed: 0', 'user', 'rating', 'comment', 'ID', 'name'], dtype='object')

In [279]:
df.head()

,Unnamed: 0,user,rating,comment,ID,name
0,0,Torsten,10.0,NaN,30549.0,Pandemic
1,1,mitnachtKAUBO-I,10.0,Hands down my favorite new game of BGG CON 200...,30549.0,Pandemic
2,2,avlawn,10.0,I tend to either love or easily tire of co-op ...,30549.0,Pandemic
3,3,Mike Mayer,10.0,NaN,30549.0,Pandemic
4,4,Mease19,10.0,This is an amazing co-op game. I play mostly ...,30549.0,Pandemic


In [280]:
df['comment'].isna().sum()

10197732

In [281]:
# there are 1,100 unique games in the original dataset

df['ID'].nunique()

1100

In [306]:
dropped = df.dropna(subset=['comment'])

In [307]:
dropped['ID'].nunique() # checking count of unique games after dropping null comment rows

1100

In [308]:
dropped['comment'].isna().sum() # no nulls in comment column left
dropped.shape[0] # num rows

1880805

In [309]:
dropped['comm_len'] = dropped['comment'].str.len() # count of chars of comment
dropped = dropped[dropped['comm_len'] > 500]

dropped.shape[0], dropped['ID'].nunique()

C:\WINDOWS\TEMP/ipykernel_8112/3477455906.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dropped['comm_len'] = dropped['comment'].str.len() # count of chars of comment


(162709, 1100)

In [310]:
dropped['num_comments'] = 1
dropped = dropped [['ID', 'comm_len', 'num_comments', 'comment']]

In [311]:
dropped.head()

,ID,comm_len,num_comments,comment
5,30549.0,1235,1,Hey! I can finally rate this game I've been pl...
9,30549.0,502,1,"Fun, fun game. Strategy is required, but defin..."
17,30549.0,1349,1,When this game first came out I followed it cl...
29,30549.0,892,1,Pre-order: Boards & Bits I actually have the ...
66,30549.0,751,1,I originally rated this as a 9 and had to rais...


In [312]:
# dropped['ID'] = dropped['ID'].astype(int)
counts = dropped.groupby('ID').agg('count')

counts.reset_index(inplace=True)
counts = counts[['ID','num_comments']]
counts.sort_values('num_comments')

,ID,num_comments
710,157789.0,1
752,168917.0,1
136,1410.0,2
281,19841.0,3
654,145189.0,3
...,...,...
345,30549.0,1009
781,174430.0,1025
373,36218.0,1079
748,167791.0,1160


In [313]:
over_100 = counts[counts['num_comments'] > 10]
over_100['ID'].nunique()

1063

In [314]:
merged = over_100.merge(dropped, how='left', left_on='ID', right_on='ID')

In [315]:
merged

,ID,num_comments_x,comm_len,num_comments_y,comment
0,1.0,222,1080,1,This is simply my favorite game of all time. ...
1,1.0,222,727,1,2004 Impressions are good after first 4-player...
2,1.0,222,528,1,Box structurally sound but dirty with a little...
3,1.0,222,1844,1,I have now played this game a number of times....
4,1.0,222,619,1,"Reprint just arrived from Valley Games, and I'..."
...,...,...,...,...,...
162468,329465.0,89,1246,1,update: stay away from the collectors editio...
162469,329465.0,89,803,1,Boring repetition of playing a card for margin...
162470,329465.0,89,506,1,"It looks good, but the gameplay is really simp..."
162471,329465.0,89,727,1,Way too much reading. Especially when most of ...


In [316]:
merged.rename(columns={'num_comments_x':'gameID_total_comments'}, inplace=True)
merged = merged[['ID', 'gameID_total_comments', 'comment']]
merged.head()

,ID,gameID_total_comments,comment
0,1.0,222,This is simply my favorite game of all time. ...
1,1.0,222,2004 Impressions are good after first 4-player...
2,1.0,222,Box structurally sound but dirty with a little...
3,1.0,222,I have now played this game a number of times....
4,1.0,222,"Reprint just arrived from Valley Games, and I'..."


In [318]:
merged['ID'] = merged['ID'].astype(int)

C:\WINDOWS\TEMP/ipykernel_8112/3997642578.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged['ID'] = merged['ID'].astype(int)


In [319]:
merged.to_csv('repo/filtered_reviews.csv')